In [1]:
import pandas as pd

In [ ]:
encounter = pd.read_parquet("../dataset/encounter.parquet")
encounter["encounterYear"] = encounter["encounterTs"].dt.year
total_encounter = encounter.groupby("encounterYear").size().reset_index(name="count")
model_encounter = (
    encounter[encounter["prediction"] == encounter["readmissionStatus"]]
    .groupby("encounterYear")
    .size()
    .reset_index(name="count")
)

# Calculate LACE accuracy per year
lace_encounter = (
    encounter[encounter["lace_prediction"] == encounter["readmissionStatus"]]
    .groupby("encounterYear")
    .size()
    .reset_index(name="count")
)

# Encounter Left join
result = total_encounter.merge(model_encounter, on="encounterYear", how="left", suffixes=("", "_model"))
result = result.merge(lace_encounter, on="encounterYear", how="left", suffixes=("", "_lace"))
# Calculate percentages
result["model_prediction"] = round(result["count_model"] * 100 / result["count"], 2)
result["lace_prediction"] = round(result["count_lace"] * 100 / result["count"], 2)

result = result[["encounterYear", "model_prediction", "lace_prediction"]].sort_values("encounterYear", ascending=False)

display(result)

,encounterYear,model_prediction,lace_prediction
10,2200,69.45,49.59
9,2199,68.13,46.22
8,2198,70.34,46.71
7,2197,70.99,46.38
6,2196,70.57,46.24
5,2195,70.86,45.63
4,2194,71.65,46.23
3,2193,71.72,47.11
2,2192,71.23,46.48
1,2191,70.96,45.28
